In [ ]:
class TruncatedGaussianHMCSampler(TruncatedGaussianSamplerBase):
    def __init__(self, mean, cov, lower, upper, step_size=0.01, n_steps=10, key=None):
        super().__init__(mean, cov, lower, upper, key)
        self.step_size = step_size
        self.n_steps = n_steps

    def sample(self, n_samples, burn_in=100, x0=None):
        if x0 is None:
            x0 = self._initial_point()

        def reflect(x, p):
            for i in range(self.d):
                out_lo = x[i] < self.lower[i]
                out_hi = x[i] > self.upper[i]

                x = x.at[i].set(jnp.where(out_lo, 2*self.lower[i] - x[i], x[i]))
                x = x.at[i].set(jnp.where(out_hi, 2*self.upper[i] - x[i], x[i]))

                p = p.at[i].set(jnp.where(out_lo | out_hi, -p[i], p[i]))

            return x, p

        def leapfrog(x, p):
            p = p - 0.5 * self.step_size * (self.P @ (x - self.mean))
            for _ in range(self.n_steps):
                x = x + self.step_size * p
                x, p = reflect(x, p)
                p = p - self.step_size * (self.P @ (x - self.mean))
            p = p + 0.5 * self.step_size * (self.P @ (x - self.mean))
            return x

        def hmc_step(x, key):
            p = random.normal(key, (self.d,))
            return leapfrog(x, p)

        @partial(jit, static_argnums=(2,))
        def _sample_impl(x0, key, n_samples):
            def body(carry, _):
                x, key = carry
                key, subkey = random.split(key)
                x = hmc_step(x, subkey)
                return (x, key), x

            (x0, key), _ = lax.scan(body, (x0, key), None, length=burn_in)
            (_, _), xs = lax.scan(body, (x0, key), None, length=n_samples)
            return xs

        return _sample_impl(x0, self.key, n_samples)


In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit, vmap, lax
from jax.scipy.linalg import cho_factor, cho_solve
from functools import partial

# Utility

def norm_ppf(u):
    return jnp.sqrt(2.0) * jax.scipy.special.erfinv(2.0 * u - 1.0)

@jit
def truncnorm_sample(key, a, b, shape=()):
    Phi = 0.5 * (1 + jax.scipy.special.erf(a / jnp.sqrt(2)))
    Phi_b = 0.5 * (1 + jax.scipy.special.erf(b / jnp.sqrt(2)))
    u = random.uniform(key, shape=shape, minval=Phi, maxval=Phi_b)
    return norm_ppf(u)

class TruncatedGaussianSamplerBase:
    def __init__(self, mean, cov, lower, upper, key=None):
        self.mean = jnp.array(mean)
        self.cov = jnp.array(cov)
        self.lower = jnp.array(lower)
        self.upper = jnp.array(upper)
        self.d = self.mean.shape[0]
        self.key = random.PRNGKey(0) if key is None else key
        self.L = jnp.linalg.cholesky(self.cov)
        cf = cho_factor(self.cov)
        self.P = cho_solve(cf, jnp.eye(self.d))

    def _initial_point(self):
        return jnp.clip(self.mean, self.lower + 1e-6, self.upper - 1e-6)

class TruncatedGaussianGibbsSampler(TruncatedGaussianSamplerBase):
    def sample(self, n_samples, burn_in=100, thinning=1, x0=None):
        if x0 is None:
            x0 = self._initial_point()

        @partial(jit, static_argnums=(2, 3, 4))
        def _sample_impl(x0, key, n_samples, burn_in, thinning):
            def step(carry, _):
                x, key = carry
                keys = random.split(key, self.d + 1)
                for i in range(self.d):
                    pii = self.P[i, i]
                    mu_i = self.mean[i] - (1.0 / pii) * (self.P[i] @ (x - self.mean))
                    std_i = jnp.sqrt(1.0 / pii)
                    a = (self.lower[i] - mu_i) / std_i
                    b = (self.upper[i] - mu_i) / std_i
                    z = truncnorm_sample(keys[i], a, b)
                    x = x.at[i].set(mu_i + std_i * z)
                return (x, keys[-1]), x

            (x0, key), _ = lax.scan(step, (x0, key), None, length=burn_in)
            (_, _), xs = lax.scan(step, (x0, key), None, length=n_samples * thinning)
            return xs[thinning - 1::thinning]

        return _sample_impl(x0, self.key, n_samples, burn_in, thinning)

class TruncatedGaussianGHKSampler(TruncatedGaussianSamplerBase):
    def sample(self, n_samples, x0=None):
        def _one_sample(key):
            zs = jnp.zeros(self.d)
            x = jnp.zeros(self.d)
            for i in range(self.d):
                key, sub = random.split(key)
                mu_partial = self.mean[i] + (self.L[i, :i] @ zs[:i])
                Li = self.L[i, i]
                a = (self.lower[i] - mu_partial) / Li
                b = (self.upper[i] - mu_partial) / Li
                z = truncnorm_sample(sub, a, b)
                zs = zs.at[i].set(z)
                x = x.at[i].set(mu_partial + Li * z)
            return x
        return vmap(_one_sample)(random.split(self.key, n_samples))

class TruncatedGaussianMinimaxTiltingSampler(TruncatedGaussianSamplerBase):
    def sample(self, n_samples, x0=None):
        @partial(jit, static_argnums=(0,))
        def _sample_impl(n_samples):
            def body(carry, _):
                samples, key = carry
                key, sub = random.split(key)
                z = random.normal(sub, (self.d,))
                x = self.mean + self.L @ z
                inside = jnp.all((x >= self.lower) & (x <= self.upper))
                return (samples, key), jnp.where(inside, x, jnp.zeros_like(x))
            (samples, _), out = lax.scan(body, (jnp.zeros((n_samples, self.d)), self.key), None, length=n_samples)
            return out
        return _sample_impl(n_samples)

class TruncatedGaussianSliceSampler(TruncatedGaussianSamplerBase):
    def sample(self, n_samples, burn_in=100, thinning=1, x0=None):
        if x0 is None:
            x0 = self._initial_point()

        @partial(jit, static_argnums=(2, 3, 4))
        def _sample_impl(x0, key, n_samples, burn_in, thinning):
            def update(carry, _):
                x, key = carry
                key, sub = random.split(key)
                logp = -0.5 * (x - self.mean) @ self.P @ (x - self.mean)
                key, u_key = random.split(key)
                logy = logp + jnp.log(random.uniform(u_key))
                for i in range(self.d):
                    var = 1.0 / self.P[i, i]
                    w = jnp.sqrt(var)
                    L = jnp.maximum(self.lower[i], x[i] - w * random.uniform(key))
                    R = jnp.minimum(self.upper[i], L + w)
                    key, sub2 = random.split(key)
                    x = x.at[i].set(random.uniform(sub2, minval=L, maxval=R))
                return (x, key), x
            (x0, key), _ = lax.scan(update, (x0, key), None, length=burn_in)
            (_, _), xs = lax.scan(update, (x0, key), None, length=n_samples * thinning)
            return xs[thinning - 1::thinning]

        return _sample_impl(x0, self.key, n_samples, burn_in, thinning)

class TruncatedGaussianHMCSampler(TruncatedGaussianSamplerBase):
    def __init__(self, mean, cov, lower, upper, step_size=0.1, n_steps=10, key=None):
        super().__init__(mean, cov, lower, upper, key)
        self.step_size = step_size
        self.n_steps = n_steps

    def sample(self, n_samples, burn_in=100, x0=None):
        if x0 is None:
            x0 = self._initial_point()

        def _hmc_step(x, key):
            key, p_key = random.split(key)
            p = random.normal(p_key, (self.d,))
            def leapfrog(carry, _):
                x, p = carry
                p = p - 0.5 * self.step_size * (self.P @ (x - self.mean))
                x = x + self.step_size * p
                x = jnp.where(x < self.lower, 2 * self.lower - x, x)
                x = jnp.where(x > self.upper, 2 * self.upper - x, x)
                p = jnp.where((x < self.lower) | (x > self.upper), -p, p)
                p = p - 0.5 * self.step_size * (self.P @ (x - self.mean))
                return (x, p), None
            (x_new, _), _ = lax.scan(leapfrog, (x, p), None, length=self.n_steps)
            return x_new, key

        @partial(jit, static_argnums=(2,))
        def _sample_impl(x0, key, n_samples):
            def body(carry, _):
                x, key = carry
                x, key = _hmc_step(x, key)
                return (x, key), x
            (x0, key), _ = lax.scan(body, (x0, key), None, length=burn_in)
            (_, _), xs = lax.scan(body, (x0, key), None, length=n_samples)
            return xs

        return _sample_impl(x0, self.key, n_samples)

class TruncatedGaussianRejectionSampler(TruncatedGaussianSamplerBase):
    def sample(self, n_samples, x0=None):
        def one_sample(key):
            z = random.normal(key, (self.d,))
            return self.mean + self.L @ z
        return vmap(one_sample)(random.split(self.key, n_samples))

class TruncatedGaussianImportanceSampler(TruncatedGaussianSamplerBase):
    def sample(self, n_samples, x0=None):
        def one_sample(key):
            z = random.normal(key, (self.d,))
            x = self.mean + self.L @ z
            w = 1.0  # placeholder
            return x, w
        return vmap(one_sample)(random.split(self.key, n_samples))

In [ ]:
def _calculate_constraint_coef(Smile_TIV):

    first_derivative = Smile_TIV.pred_deriv['first_derivative']
    second_derivative = Smile_TIV.pred_deriv['second_derivative']
    function_val = Smile_TIV.pred_deriv['function_val']

    b, a = Smile_TIV.m_scaler.inverse(1), Smile_TIV.m_scaler.inverse(0)

    knots = Smile_TIV.m_scaler.inverse(Smile_TIV.x_test_extra)
    knots_in_m = Smile_TIV.x_test_extra

    B = np.zeros(len(knots))

    for i in range(len(knots)):

        comp_1 = knots_in_m[i]**2 * first_derivative[i] / ((b-a)* 4* function_val[i]**2) 
        comp_2 = -knots_in_m[i] / function_val[i]
        comp_3 = -first_derivative[i] / ( (b-a)* 4* function_val[i])
        comp_4 = -first_derivative[i] / ( (b-a)* 16)
        
        B[i] = (comp_1 + comp_2 + comp_3 + comp_4)/(b-a)
    
    C = 1/(2*(b-a)**2)
        
    return B,C


B, C = _calculate_constraint_coef(Smile_TIV)    
    

In [ ]:
B, C = _calculate_constraint_coef(Smile_TIV)

Lambda = np.zeros((m, m))
for i in range(m):
   if i != 0 and i != m-1:        
        Lambda[i, i-1] = -B[i]/(2*delta_m) + C / (delta_m**2)
        Lambda[i, i+1] = B[i]/(2*delta_m) + C / (delta_m**2)
        Lambda[i, i] = -2*C / (delta_m**2)
        
   if i == 0:
         Lambda[i, i+1] = B[i]/(2*delta_m) + C / (delta_m**2)
         Lambda[i, i] = -C / (delta_m**2)
   if i == m-1:
         Lambda[i, i-1] = -B[i]/(2*delta_m) + C / (delta_m**2)
         Lambda[i, i] = -C / (delta_m**2)
           

In [ ]:

def plot_samples(ax, samples, lower, upper, title):
    ax.scatter(samples[:, 0], samples[:, 1], s=10, alpha=0.5)
    ax.set_xlim(lower[0] - 0.2, upper[0] + 0.2)
    ax.set_ylim(lower[1] - 0.2, upper[1] + 0.2)
    ax.set_title(title)
    ax.set_aspect('equal')

def gaussian_contour(ax, mean, cov):
    # plot 1- and 2-sigma ellipses
    from matplotlib.patches import Ellipse
    vals, vecs = np.linalg.eigh(cov)
    for nsig in [1, 2]:
        width, height = 2 * nsig * np.sqrt(vals)
        angle = np.degrees(np.arctan2(*vecs[:,0][::-1]))
        ell = Ellipse(xy=mean, width=width, height=height,
                      angle=angle, edgecolor='k', facecolor='none', lw=1)
        ax.add_patch(ell)

def main():
    # Problem setup
    mean = jnp.array([0.0, 0.0])
    cov = jnp.array([[1.0, 0.8],
                     [0.8, 1.0]])
    lower = jnp.array([-2.0, -1.5])
    upper = jnp.array([ 0.5,  1.2])
    key = random.PRNGKey(42)
    
    n_samples = 1000
    
    # Instantiate each sampler
    samplers = {
        "Gibbs":       TruncatedGaussianGibbsSampler(mean, cov, lower, upper),
        "GHK":         TruncatedGaussianGHKSampler(mean, cov, lower, upper),
        "Minimax":     TruncatedGaussianMinimaxTiltingSampler(mean, cov, lower, upper),
        "Slice":       TruncatedGaussianSliceSampler(mean, cov, lower, upper),
        "HMC":         TruncatedGaussianHMCSampler(mean, cov, lower, upper, step_size=0.1, n_steps=15),
        "Rejection":   TruncatedGaussianRejectionSampler(mean, cov, lower, upper),
        "Importance":  TruncatedGaussianImportanceSampler(mean, cov, lower, upper),
    }
    
    # Draw samples
    samples_dict = {}
    for name, sampler in samplers.items():
        key, sub = random.split(key)
        if name in ["Gibbs", "Slice"]:
            samp = sampler.sample(n_samples, burn_in=200, thinning=2)
        elif name == "HMC":
            samp = sampler.sample(n_samples, burn_in=200)      # ← no thinning
        elif name == "Minimax":
            samp = sampler.sample(n_samples)
        elif name in ["GHK", "Rejection"]:
            samp = sampler.sample(n_samples)
        else:  # Importance
            samp, weights = sampler.sample(n_samples)
            samp = np.array(samp)
        samples_dict[name] = np.array(samp)

    # Plotting
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    for ax, (name, samp) in zip(axes, samples_dict.items()):
        plot_samples(ax, samp, np.array(lower), np.array(upper), name)
        gaussian_contour(ax, np.array(mean), np.array(cov))
    
    # Turn off unused subplot
    axes[-1].axis('off')
    fig.suptitle("2D Truncated Gaussian: Samples from Different Samplers", fontsize=16)
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()

# Sampling Trials


In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

class TruncatedMVNSampler:
    def __init__(self, mu, Sigma, a, b):
        self.mu = np.asarray(mu)
        self.Sigma = np.asarray(Sigma)
        self.a = np.asarray(a)
        self.b = np.asarray(b)
        self.d = len(mu)

        assert self.Sigma.shape == (self.d, self.d), "Covariance matrix shape mismatch"
        assert self.a.shape == (self.d,) and self.b.shape == (self.d,), "Bounds shape mismatch"

        # Cholesky decomposition of covariance matrix
        self.L = np.linalg.cholesky(self.Sigma)

        # Transform bounds to standard normal space
        self.a_std = np.linalg.solve(self.L, self.a - self.mu)
        self.b_std = np.linalg.solve(self.L, self.b - self.mu)

    def transform_to_original_space(self, z):
        return self.L @ z + self.mu

    def log_tilted_density(self, z, theta):
        return -0.5 * np.dot(z, z) + np.dot(theta, z)

    def grad_cgf(self, theta, num_samples=10000):
        z = np.random.randn(num_samples, self.d)
        weights = np.exp(z @ theta - 0.5 * np.sum(z**2, axis=1))
        return (weights[:, None] * z).mean(axis=0) / weights.mean()

    def hess_cgf(self, theta, num_samples=10000):
        z = np.random.randn(num_samples, self.d)
        weights = np.exp(z @ theta - 0.5 * np.sum(z**2, axis=1))
        Z_bar = (weights[:, None] * z).mean(axis=0) / weights.mean()
        ZZ_bar = (weights[:, None, None] * np.einsum('ni,nj->nij', z, z)).mean(axis=0) / weights.mean()
        return ZZ_bar - np.outer(Z_bar, Z_bar)

    def find_optimal_tilting(self):
        def objective(theta):
            grad = self.grad_cgf(theta)
            return 0.5 * np.dot(grad, grad)

        def grad_objective(theta):
            return self.hess_cgf(theta) @ self.grad_cgf(theta)

        theta0 = np.zeros(self.d)
        res = minimize(objective, theta0, jac=grad_objective, method='BFGS')
        return res.x

    def sample(self, n_samples=1):
        theta_star = self.find_optimal_tilting()
        accepted = []

        while len(accepted) < n_samples:
            z = np.random.randn(self.d)
            log_w = -np.dot(theta_star, z) + 0.5 * np.dot(theta_star, theta_star)
            if np.all(z >= self.a_std) and np.all(z <= self.b_std):
                u = np.random.rand()
                if np.log(u) < -log_w:
                    accepted.append(self.transform_to_original_space(z))

        return np.array(accepted) if n_samples > 1 else accepted[0]

In [ ]:
import numpy as np
from scipy.stats import norm

params = constrained_gp.eta_parameters

mean = params['mean_eta']
cov = params['cov_eta']
lower = params['lower_eta']
upper = params['upper_eta']

std_devs = np.sqrt(np.diag(cov))
marginal_masses = []

for i in range(len(mean)):
	mu = mean[i]
	sigma = std_devs[i]
	a = (lower[i] - mu) / sigma
	b = (upper[i] - mu) / sigma
	prob_mass = norm.cdf(b) - norm.cdf(a)
	marginal_masses.append(prob_mass)


/tmp/ipykernel_946006/464037173.py:11: RuntimeWarning: invalid value encountered in sqrt
  std_devs = np.sqrt(np.diag(cov))


In [ ]:
samplerr = TruncatedMVnGHKTiltedSampler2(mean,
                              cov+ np.eye(len(mean))*1e-3,
                              lower,
                              upper)

In [ ]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

class TruncatedMVnGHKTiltedSampler2:
	def __init__(self, mu, Sigma, a, b):
		self.mu = np.asarray(mu)
		self.Sigma = np.asarray(Sigma)
		self.a = np.asarray(a)
		self.b = np.asarray(b)
		self.d = len(mu)

		assert self.Sigma.shape == (self.d, self.d), "Covariance matrix shape mismatch"
		assert self.a.shape == (self.d,) and self.b.shape == (self.d,), "Bounds shape mismatch"

		# Cholesky decomposition of covariance matrix
		self.L = np.linalg.cholesky(self.Sigma)

		# Transform bounds to standard normal space
		self.a_std = np.linalg.solve(self.L, self.a - self.mu)
		self.b_std = np.linalg.solve(self.L, self.b - self.mu)

	def grad_cgf(self, theta, num_samples=10000):
		z = np.random.randn(num_samples, self.d)
		weights = np.exp(z @ theta - 0.5 * np.sum(z**2, axis=1))
		return (weights[:, None] * z).mean(axis=0) / weights.mean()

	def hess_cgf(self, theta, num_samples=10000):
		z = np.random.randn(num_samples, self.d)
		weights = np.exp(z @ theta - 0.5 * np.sum(z**2, axis=1))
		Z_bar = (weights[:, None] * z).mean(axis=0) / weights.mean()
		ZZ_bar = (weights[:, None, None] * np.einsum('ni,nj->nij', z, z)).mean(axis=0) / weights.mean()
		return ZZ_bar - np.outer(Z_bar, Z_bar)

	def find_optimal_tilting(self):
		def objective(theta):
			grad = self.grad_cgf(theta)
			return 0.5 * np.dot(grad, grad)

		def grad_objective(theta):
			return self.hess_cgf(theta) @ self.grad_cgf(theta)

		theta0 = np.zeros(self.d)
		res = minimize(objective, theta0, jac=grad_objective, method='BFGS')
		return res.x

	def sample(self, n_samples=1):
		theta_star = self.find_optimal_tilting()
		print(theta_star)
		samples = []
		log_weights = []

		for _ in range(n_samples):
			z = np.zeros(self.d)
			log_weight = 0.0
			for i in range(self.d):
				Li = self.L[i, :i]
				mu_i = np.dot(Li, z[:i])
				std_i = self.L[i, i]

				lower = (self.a_std[i] - mu_i) / std_i
				upper = (self.b_std[i] - mu_i) / std_i

				theta_i = theta_star[i] * std_i
				alpha = norm.cdf(upper - theta_i) - norm.cdf(lower - theta_i)
				print(alpha)
				u = np.random.rand()
				quantile = norm.ppf(norm.cdf(lower - theta_i) + u * alpha)
				z_i = quantile + theta_i

				z[i] = mu_i + std_i * z_i
				log_weight += -theta_star[i] * z_i + 0.5 * theta_star[i]**2

			x = self.L @ z + self.mu
			samples.append(x)
			log_weights.append(log_weight)

		return np.array(samples) if n_samples > 1 else samples[0], np.exp(-np.array(log_weights))


# Kaldiracaklar

In [ ]:
# Enable NumPy <-> R auto-conversion
numpy2ri.activate()

# Import the tmvtnorm package
tmvtnorm = importr("tmvtnorm")

# Use your own pre-defined NumPy arrays:
# mean, cov, lower, upper must already be defined

# Wrap for R
mean_r = FloatVector(constrained_gp.eta_parameters['mean_eta'].tolist())
mod_r = FloatVector(constrained_gp.eta_parameters['mean_star_eta'].tolist())
lower_r = FloatVector(constrained_gp.lower_bound.tolist())
upper_r = FloatVector(constrained_gp.upper_bound.tolist())

def to_r_matrix(array):
	return robjects.r.matrix(FloatVector(array.flatten('F')), nrow=array.shape[0])

new_cov = constrained_gp.eta_parameters['cov_eta'] + np.eye(constrained_gp.eta_parameters['cov_eta'].shape[0]) * 1e-6
new_cov = 0.5 * (new_cov + new_cov.T)  # Ensure symmetry

#new_cov += 1e-3 * np.eye(new_cov.shape[0])
cov_r = to_r_matrix(cov_eta)

# Number of samples
n_samples = 1000

# Call R function
samples_r = tmvtnorm.rtmvnorm(
	n=n_samples,
	mean=mean_r,
	sigma=cov_r,
	lower=lower_r,
	upper=upper_r,
	algorithm = "gibbs",
	**{'start.value': mod_r}
)


In [ ]:
def _calculate_constraint_coef(Smile_TIV):

    first_derivative = Smile_TIV.pred_deriv['first_derivative']
    second_derivative = Smile_TIV.pred_deriv['second_derivative']
    function_val = Smile_TIV.pred_deriv['function_val']

    b, a = Smile_TIV.m_scaler.inverse(1), Smile_TIV.m_scaler.inverse(0)

    knots = Smile_TIV.m_scaler.inverse(Smile_TIV.x_test_extra)
    knots_in_m = Smile_TIV.x_test_extra

    B = np.zeros(len(knots))

    for i in range(len(knots)):

        comp_1 = knots_in_m[i]**2 * first_derivative[i] / ((b-a)* 4* function_val[i]**2) 
        comp_2 = -knots_in_m[i] / function_val[i]
        comp_3 = -first_derivative[i] / ( (b-a)* 4* function_val[i])
        comp_4 = -first_derivative[i] / ( (b-a)* 16)
        
        B[i] = (comp_1 + comp_2 + comp_3 + comp_4)/(b-a)
    
    C = 1/(2*(b-a)**2)
        
    return B,C


B, C = _calculate_constraint_coef(Smile_TIV)    
    

In [ ]:
B, C = _calculate_constraint_coef(Smile_TIV)

Lambda = np.zeros((m, m))
for i in range(m):
   if i != 0 and i != m-1:        
        Lambda[i, i-1] = -B[i]/(2*delta_m) + C / (delta_m**2)
        Lambda[i, i+1] = B[i]/(2*delta_m) + C / (delta_m**2)
        Lambda[i, i] = -2*C / (delta_m**2)
        
   if i == 0:
         Lambda[i, i+1] = B[i]/(2*delta_m) + C / (delta_m**2)
         Lambda[i, i] = -C / (delta_m**2)
   if i == m-1:
         Lambda[i, i-1] = -B[i]/(2*delta_m) + C / (delta_m**2)
         Lambda[i, i] = -C / (delta_m**2)
           